In [4]:
import pandas as pd
import requests
from datetime import date, timedelta
import matplotlib.pyplot as plt


In [19]:
def fetch_precip_hourly_openmeteo(lat: float, lon: float, start: date, end: date) -> pd.DataFrame:
    """
    Baixa precipitação horária (mm) da API histórica da Open-Meteo
    e retorna DataFrame com colunas:
    ['time','precipitation'] no intervalo [start, end].
    """
    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start.isoformat()}&end_date={end.isoformat()}"
        "&hourly=precipitation&timezone=auto"
    )
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    j = r.json()
    hourly = j.get("hourly", {})
    df = pd.DataFrame(hourly)
    if df.empty:
        raise ValueError("Sem dados retornados pela API.")
    df["time"] = pd.to_datetime(df["time"])
    return df


def build_daily_rain_df(lat: float, lon: float, days_back: int = 3650) -> pd.DataFrame:
    """
    Constrói DataFrame diário com:
      - 'ds' = data (diária)
      - 'y'  = chuva acumulada do dia (mm)
    """
    end = date.today() - timedelta(days=1)           # até ontem
    start = end - timedelta(days=days_back-1)        # janela histórica

    df_h = fetch_precip_hourly_openmeteo(lat, lon, start, end)
    df_h["date"] = df_h["time"].dt.date

    # soma a chuva por dia
    df_daily = df_h.groupby("date", as_index=False)["precipitation"].sum()
    df_daily.rename(columns={"date":"ds","precipitation":"y"}, inplace=True)

    # Garantir datas contínuas (preencher zero onde faltou)
    full_idx = pd.date_range(start=start, end=end, freq="D").date
    df_full = pd.DataFrame({"ds": full_idx})
    df = df_full.merge(df_daily, on="ds", how="left").fillna({"y": 0.0})

    # Deixar no tipo certo
    df["ds"] = pd.to_datetime(df["ds"])
    df["y"] = df["y"].clip(lower=0)  # segurança

    return df


In [20]:
# Exemplo: região de Goiânia (ajuste depois para a fazenda que você quiser)
LAT, LON = -16.68, -49.25

history_df = build_daily_rain_df(LAT, LON, days_back=3650)
history_df.head(), history_df.tail()


(          ds    y
 0 2015-11-17  3.8
 1 2015-11-18  5.7
 2 2015-11-19  1.8
 3 2015-11-20  0.5
 4 2015-11-21  4.6,
              ds    y
 3645 2025-11-09  3.5
 3646 2025-11-10  0.7
 3647 2025-11-11  4.6
 3648 2025-11-12  1.2
 3649 2025-11-13  0.1)

In [35]:
import pandas as pd

def create_rain_features(df, target_col="y", date_col=None):
    """
    Cria features de série temporal (lags, janelas, calendário)
    a partir de um DataFrame diário com chuva.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame com uma coluna de chuva (target_col) e
        ou uma coluna de datas (date_col) ou índice datetime.
    target_col : str
        Nome da coluna de chuva (default = 'y').
    date_col : str ou None
        Nome da coluna de datas. Se None, usa o índice como data.

    Retorna
    -------
    df_features : pd.DataFrame
        DataFrame com features e a coluna alvo, sem NaN.
    """
    df = df.copy()

    # --- Garantir coluna de data 'ds' ---
    if date_col is not None:
        df["ds"] = pd.to_datetime(df[date_col])
        df = df.set_index("ds")
    else:
        # usa o índice como data
        df.index = pd.to_datetime(df.index)
        df["ds"] = df.index

    # Garantir que está em frequência diária (importante para lags)
    df = df.asfreq("D")

    # Garantir tipo float na chuva
    df[target_col] = df[target_col].astype(float).fillna(0.0)

    # 1. Lags (passado recente)
    df["lag_1"] = df[target_col].shift(1)   # chuva de ontem
    df["lag_2"] = df[target_col].shift(2)   # 2 dias atrás
    df["lag_7"] = df[target_col].shift(7)   # 1 semana atrás

    # 2. Janelas móveis (tendência recente)
    # shift(1) evita olhar o valor do próprio dia
    df["rolling_mean_7"]  = df[target_col].shift(1).rolling(window=7).mean()
    df["rolling_sum_7"]   = df[target_col].shift(1).rolling(window=7).sum()
    df["rolling_mean_30"] = df[target_col].shift(1).rolling(window=30).mean()

    # 3. Calendário (sazonalidade)
    df["month"]       = df["ds"].dt.month
    df["day_of_year"] = df["ds"].dt.dayofyear
    df["day_of_week"] = df["ds"].dt.dayofweek

    # Remover as primeiras linhas com NaN por causa de lags/janelas
    df_features = df.dropna().copy()

    return df_features


In [37]:
df_features = create_rain_features(history_df, target_col="y")
df_features.head()


,ds,y,lag_1,lag_2,lag_7,rolling_mean_7,rolling_sum_7,rolling_mean_30,month,day_of_year,day_of_week


In [1]:
!python -m pip install tensorflow scikit-learn joblib


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
